### Second Step - Time-Series Prediction using Transformer

In [1]:
import time
import math
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler

from utils.ML import time_series_dataframe, make_dataframe
from utils.DL import new_multistep_time_series, plot_and_loss2
from utils.DL import train_tmp, evaluate2
from utils.DL import TransAm

In [7]:
# Hyper Parameters
input_window = 240
output_window = 120
epochs = 500
batch_size = 1024
lr = 0.001

device = 'cuda' if torch.cuda.is_available else 'cpu'

In [3]:
##### Load Dataset #####
print(f'>>> Load Dataset!')

df_DL, _, _ = time_series_dataframe('./temp_add_gps/')

df_DL_temp = df_DL['TEMP'].values
df_DL_label = df_DL['label'].values

# Preprocessing
scaler_train = MinMaxScaler()
scaler_test = MinMaxScaler()

train_len = int(len(df_DL) * 0.8)

train_data = df_DL_temp[:train_len]
train_label = df_DL_label[:train_len]
test_data = df_DL_temp[train_len:]
test_label = df_DL_label[train_len:]

train_data = scaler_train.fit_transform(train_data.reshape(-1,1)).reshape(-1)
test_data = scaler_test.fit_transform(test_data.reshape(-1,1)).reshape(-1)

train_data, _ = new_multistep_time_series(train_data, train_label, input_window, output_window)
test_data, test_label = new_multistep_time_series(test_data, test_label, input_window, output_window)

train_data = train_data.to(device)
test_data = test_data.to(device)

print(f'X_train Shape : {train_data.shape}')
print(f'X_test Shape : {test_data.shape}')
print(f'>>> Successfully load dataset!')

>>> Load Dataset!
X_train Shape : torch.Size([529564, 2, 240])
X_test Shape : torch.Size([132122, 2, 240])
>>> Successfully load dataset!


In [4]:
##### Load Model ######
print(f'>>> Load Model!')

model = TransAm().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(params = model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.98)

print(f'>>> Successfully load model!')

>>> Load Model!
>>> Successfully load model!


In [ ]:
##### Train & Test #####
best_val_loss = float("inf")
best_model = None

for epoch in range(1, epochs + 1):
    start_time = time.time()
    train_tmp(model, train_data, batch_size, optimizer, criterion, input_window, output_window, epoch, scheduler)
    
    if (epoch % 20 == 0):
        truth, test_result, result_to_ML, val_loss = plot_and_loss2(model, test_data, criterion,input_window, output_window, scaler_test)
    else:
        val_loss = evaluate2(model, test_data, criterion, output_window, input_window)
    
    print('-' * 90)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.5f} | valid ppl {:8.2f} |'.format(epoch, (time.time() - start_time),
                                        val_loss, math.exp(val_loss)))
    print('-' * 90)
    
    scheduler.step()
    

PATH = "./checkpoints/"
torch.save(model, PATH + 'prediction_model.pt')
torch.save(model.state_dict(), PATH + 'prediction_model_state_dict.pt')